In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from typing import Dict, Tuple
import keras.layers as l
from keras import models, callbacks, utils, losses

In [2]:
text = ''
with open('harry_potter.txt', 'r') as file:
    text = file.read()

def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 100

alphabet = np.array(sorted(set(text)))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[char] for char in text])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)


In [3]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.SimpleRNN(512, return_sequences=True, stateful=True),
    l.SimpleRNN(512, return_sequences=True, stateful=True),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=20, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/20
5/5 [==============================] - 3s 359ms/step - loss: 5.4671 - accuracy: 0.0406
Epoch 2/20
5/5 [==============================] - 2s 353ms/step - loss: 4.0479 - accuracy: 0.1281
Epoch 3/20
5/5 [==============================] - 2s 345ms/step - loss: 3.4092 - accuracy: 0.1269
Epoch 4/20
5/5 [==============================] - 2s 345ms/step - loss: 3.3715 - accuracy: 0.1363
Epoch 5/20
5/5 [==============================] - 2s 343ms/step - loss: 3.3526 - accuracy: 0.1363
Epoch 6/20
5/5 [==============================] - 2s 341ms/step - loss: 3.3446 - accuracy: 0.1363
Epoch 7/20
5/5 [==============================] - 2s 348ms/step - loss: 3.3368 - accuracy: 0.1363
Epoch 8/20
5/5 [==============================] - 2s 345ms/step - loss: 3.3327 - accuracy: 0.1363
Epoch 9/20
5/5 [==============================] - 2s 342ms/step - loss: 3.3308 - accuracy: 0.1363
Epoch 10/20
5/5 [==============================] - 2s 354ms/step - loss: 3.3278 - accuracy: 0.1363
Epoch 11/20
5/5 [==

In [4]:
def predict_next(sample: str, model: keras.Sequential, tokenizer: Dict[str, int], vocabulary: Dict[int, str], n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [5]:
print(predict_next(
    sample='Гарри',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=200,
    temperature=0.6,
    batch_size=BATCH_SIZE
))

Гарри а о и  а е зо двл мо ечдоноло ч а о и лоепо и  и е,  ло оне
нс та натланетьеро ре а сагЕни аалунне ла о ннете ско ирмуы у о о и ,ливирилра о а а о ате
а логн дноле и р, у зотми риль о ино о не мне ии


In [6]:
print(predict_next(
    sample='Магия',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=100,
    temperature=0.2,
    batch_size=BATCH_SIZE
))

Магия а о а но а и но о  о а на а а  а о и е и  о о  а а  о е а и  и а о  о а и и  о  но о  о ро а  о о а


In [7]:
print(predict_next(
    sample='МакГонагалл',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=150,
    temperature=0.81,
    batch_size=BATCH_SIZE
))

МакГонагалленила з ини я, угискь кери сакаланивнии ора ро о лс к рнт—азигялрту — ноо енино ь-.генмнжрло сое ори е Гажьремосооана а а 
о оо
ов 
ащва бку бе к вмаы
